In [1]:
import pandas as pd
import numpy as np

In [2]:
datalake = pd.read_csv('./datalake/datalake.csv', encoding='utf-8', header=0, low_memory=False)

In [3]:
original_data = datalake.copy(deep=True)
print(datalake.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850013 entries, 0 to 850012
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   COMISARIA           850013 non-null  object
 1   CUADRA              53118 non-null   object
 2   DERIVADA_FISCALIA   715460 non-null  object
 3   DIRECCION           850013 non-null  object
 4   DIST_CIA            850013 non-null  object
 5   DIST_HECHO          850013 non-null  object
 6   DPTO_CIA            850013 non-null  object
 7   DPTO_HECHO          850013 non-null  object
 8   EDAD                847713 non-null  object
 9   ESTADO_DEN          850013 non-null  object
 10  EST_CIVIL           849957 non-null  object
 11  FECHA_HORA_HECHO    850013 non-null  object
 12  ID_COMISARIA        850013 non-null  int64 
 13  ID_EST_CIVIL        849962 non-null  object
 14  ID_LIBRO            850013 non-null  object
 15  ID_MATERIA          850013 non-null  int64 
 16  ID

In [4]:
# Idenficando las columnas por tipo de dato
object_columns = []
int64_columns = []
def show_types():
    global object_columns, int64_columns
    object_columns = datalake.select_dtypes(['object']).columns.to_list()
    int64_columns = datalake.select_dtypes(['int64']).columns.to_list()
    print(f'object\n {object_columns}')
    print(f'int64\n {int64_columns}')
show_types()

object
 ['COMISARIA', 'CUADRA', 'DERIVADA_FISCALIA', 'DIRECCION', 'DIST_CIA', 'DIST_HECHO', 'DPTO_CIA', 'DPTO_HECHO', 'EDAD', 'ESTADO_DEN', 'EST_CIVIL', 'FECHA_HORA_HECHO', 'ID_EST_CIVIL', 'ID_LIBRO', 'ID_NIVEL_EDUCATIVO', 'ID_SIT_PERSONA', 'ID_TIPO_DENUNCIA', 'LIBRO', 'MATERIA', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'PROV_CIA', 'PROV_HECHO', 'REGION', 'SEXO', 'SIT_PERSONA', 'SUB_TIPO', 'TIPO', 'TIPO_DENUNCIA', 'UBICACION', 'VIA', 'fec_registro', 'pais_natal']
int64
 ['ID_COMISARIA', 'ID_MATERIA', 'ID_MODALIDAD', 'ID_REGION', 'ID_SUBTIPO', 'ID_TIPO', 'UBIGEO_CIA', 'UBIGEO_HECHO']


In [5]:
#Se están clasificando columnas de id's (incluyendo UBIGEOs) como int64.
#Estos datos no son propiamente numéricos, de hecho, la única categoría numérica sería 'EDAD', pero ahora es de tipo object.
#Primero se convertirán todas las columnas id detectadas como int64 a object
for col in int64_columns:
    datalake[col] = datalake[col].astype(str)
show_types()

object
 ['COMISARIA', 'CUADRA', 'DERIVADA_FISCALIA', 'DIRECCION', 'DIST_CIA', 'DIST_HECHO', 'DPTO_CIA', 'DPTO_HECHO', 'EDAD', 'ESTADO_DEN', 'EST_CIVIL', 'FECHA_HORA_HECHO', 'ID_COMISARIA', 'ID_EST_CIVIL', 'ID_LIBRO', 'ID_MATERIA', 'ID_MODALIDAD', 'ID_NIVEL_EDUCATIVO', 'ID_REGION', 'ID_SIT_PERSONA', 'ID_SUBTIPO', 'ID_TIPO', 'ID_TIPO_DENUNCIA', 'LIBRO', 'MATERIA', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'PROV_CIA', 'PROV_HECHO', 'REGION', 'SEXO', 'SIT_PERSONA', 'SUB_TIPO', 'TIPO', 'TIPO_DENUNCIA', 'UBICACION', 'UBIGEO_CIA', 'UBIGEO_HECHO', 'VIA', 'fec_registro', 'pais_natal']
int64
 []


### Selección de atributos relevantes

In [6]:
relevant_columns = ['DPTO_CIA', 'DIST_CIA', 'PROV_CIA', 'DERIVADA_FISCALIA', 'EDAD', 'EST_CIVIL', 'fec_registro', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'SEXO', 'SIT_PERSONA']
datalake = datalake[relevant_columns]
datalake.rename(columns={'fec_registro': 'FEC_REGISTRO'}, inplace=True)
datalake = datalake.loc[datalake['SIT_PERSONA'] == 'DENUNCIADO']
datalake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422522 entries, 0 to 850010
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DPTO_CIA           422522 non-null  object
 1   DIST_CIA           422522 non-null  object
 2   PROV_CIA           422522 non-null  object
 3   DERIVADA_FISCALIA  349558 non-null  object
 4   EDAD               420901 non-null  object
 5   EST_CIVIL          422515 non-null  object
 6   FEC_REGISTRO       422522 non-null  object
 7   MODALIDAD          422522 non-null  object
 8   NIVEL_EDUCATIVO    16630 non-null   object
 9   OCUPACION          166714 non-null  object
 10  SEXO               422279 non-null  object
 11  SIT_PERSONA        422522 non-null  object
dtypes: object(12)
memory usage: 41.9+ MB


In [7]:
show_types()

object
 ['DPTO_CIA', 'DIST_CIA', 'PROV_CIA', 'DERIVADA_FISCALIA', 'EDAD', 'EST_CIVIL', 'FEC_REGISTRO', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'SEXO', 'SIT_PERSONA']
int64
 []


#### Exploración de datos

##### Valores perdidos

In [8]:
# Trimea los textos y si resulta vacio, se pone nan
for col in object_columns:
    datalake[col] = datalake[col].str.strip()
datalake = datalake.replace('', np.nan)

In [9]:
# Resumen de valores perdidos
def missing_values():
    row_count = datalake.shape[0]
    print(f'Número de registros: {row_count}')
    global_null_count = datalake.isnull().sum().sum()
    print(f'Valores perdidos: {global_null_count}')
    for col in datalake.columns:
        null_count = datalake[col].isnull().sum()
        null_percent = null_count / row_count * 100
        if null_count > 0:
            print(f'{col}: {null_count} ({null_percent:.2f}%)')
missing_values()

Número de registros: 422522
Valores perdidos: 779356
DERIVADA_FISCALIA: 72967 (17.27%)
EDAD: 5808 (1.37%)
EST_CIVIL: 7 (0.00%)
FEC_REGISTRO: 16091 (3.81%)
NIVEL_EDUCATIVO: 405892 (96.06%)
OCUPACION: 278348 (65.88%)
SEXO: 243 (0.06%)


In [10]:
def getUniques(col):
    uniques = datalake[col].unique()
    print(f'{col} [{len(uniques)} unique values]')
    print(uniques)

In [11]:
for col in datalake.columns:
    getUniques(col)

DPTO_CIA [26 unique values]
['LA LIBERTAD' 'CALLAO' 'LIMA' 'AREQUIPA' 'ICA' 'ANCASH' 'LAMBAYEQUE'
 'PUNO' 'MOQUEGUA' 'APURIMAC' 'CUSCO' 'HUANCAVELICA' 'CAJAMARCA' 'PIURA'
 'JUNIN' 'TACNA' 'TUMBES' 'UCAYALI' 'LORETO' 'PASCO' 'AMAZONAS' 'HUANUCO'
 'SAN MARTIN' 'AYACUCHO' 'MADRE DE DIO' 'MADRE DE DIOS']
DIST_CIA [795 unique values]
['HUANCHACO' 'CALLAO' 'LURIGANCHO - CHOSICA' 'OCOÑA' 'PARCONA' 'PUCUSANA'
 'LA ESPERANZA' 'CHIMBOTE' 'VILLA EL SALVADOR' 'SAN MIGUEL'
 'JOSE LEONARDO ORTIZ' 'JULIACA' 'VILLA MARIA DEL TRIUNFO' 'HUAURA'
 'MAJES' 'SAMEGUA' 'JOSE LUIS BUSTAMANTE Y RIVERO' 'ATE'
 'SAN JUAN DE LURIGANCHO' 'CAYMA' 'BARRANCO' 'MIRAFLORES'
 'SAN MARTIN DE PORRES' 'VENTANILLA' 'EL AGUSTINO' 'LURIN' 'CHICLAYO'
 'ABANCAY' 'LAMBAYEQUE' 'ISLAY' 'SANTA ANITA' 'CUSCO' 'HUANCAVELICA'
 'CERRO COLORADO' 'NUEVO CHIMBOTE' 'SAN JUAN DE MIRAFLORES' 'SANTIAGO'
 'TAMBURCO' 'PUNO' 'INDEPENDENCIA' 'CAJAMARCA' 'SANTIAGO DE SURCO' 'PIURA'
 'LOS OLIVOS' 'LA MOLINA' 'SAN VICENTE DE CAÑETE' 'LA OROYA' 'LIMA'

##### Valores atípicos

###### Edad

In [12]:
# Llena los valores perdidos con 0
datalake['EDAD'] = datalake['EDAD'].fillna(0)
# Para convertir a int64 primero se convierte a float
datalake['EDAD'] = datalake['EDAD'].astype(float)
datalake['EDAD'] = datalake['EDAD'].astype(np.int64)

In [13]:
# Valores máximos y mínimos
print(datalake['EDAD'].min(), datalake['EDAD'].max())

0 728


In [14]:
# Vista ordenada de edades registradas
age_sorted = datalake.sort_values('EDAD')['EDAD']
print(age_sorted.head())
print(age_sorted.tail())
# Obtención de métricas
median = age_sorted.median()
mode = age_sorted.mode()[0]
print(f'Mediana: {median}')
print(f'Moda: {mode}')

790177    0
162763    0
63659     0
92007     0
323474    0
Name: EDAD, dtype: int64
408108    545
613740    617
176833    617
655314    627
179016    728
Name: EDAD, dtype: int64
Mediana: 34.0
Moda: 31


In [15]:
# Suplantamos los valores atípicos con la moda
min_age = 18
max_age = 75
datalake.loc[(datalake['EDAD'] < min_age) | (datalake['EDAD'] > max_age) | (datalake['EDAD'] == 0), 'EDAD'] = mode
# La columna EDAD ya no tiene valores perdidos

###### Fecha

In [16]:
import xlrd

# De fecha de excel (43677.4) a yyyy-mm-dd
def transformExcelDate(excelDate):
    dateISO = xlrd.xldate.xldate_as_datetime(float(excelDate), 0).date().isoformat()
    # print(dateISO, type(dateISO))
    return dateISO

In [17]:
# RegExp de fecha de excel
import re
excelDate = r'\d+(\.\d+)?'

In [18]:
# Función para estandarizar las fechas de los registros
def transformDate(date):
    try:
        if not pd.isna(date):
            if re.fullmatch(excelDate, date):
                return transformExcelDate(date)
            else:
                return pd.to_datetime(date).date().isoformat()
    except Exception as e:
        print(e, date)

In [19]:
datalake['FEC_REGISTRO'] = datalake['FEC_REGISTRO'].apply(transformDate)

In [20]:
# ¿Cómo llenar las fechas faltantes? las eliminamos :)
datalake = datalake.loc[datalake['FEC_REGISTRO'].isna() == False]
missing_values()

Número de registros: 406431
Valores perdidos: 725404
DERIVADA_FISCALIA: 68379 (16.82%)
EST_CIVIL: 7 (0.00%)
NIVEL_EDUCATIVO: 390328 (96.04%)
OCUPACION: 266447 (65.56%)
SEXO: 243 (0.06%)


In [21]:
show_types()

object
 ['DPTO_CIA', 'DIST_CIA', 'PROV_CIA', 'DERIVADA_FISCALIA', 'EST_CIVIL', 'FEC_REGISTRO', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'SEXO', 'SIT_PERSONA']
int64
 ['EDAD']


#### Completando valores perdidos en atributos categóricos

In [22]:
def fill_equally(col):
    datalake[col] = datalake[col].fillna(pd.Series(np.random.choice(datalake[col].dropna().unique(), size=datalake.shape[0])))

In [23]:
fill_equally('NIVEL_EDUCATIVO')
fill_equally('OCUPACION')
fill_equally('DERIVADA_FISCALIA')

In [24]:
datalake.reset_index(drop=True, inplace=True)
print(datalake.head())
print(datalake.tail())
print(missing_values())

      DPTO_CIA              DIST_CIA  PROV_CIA   DERIVADA_FISCALIA  EDAD  \
0  LA LIBERTAD             HUANCHACO  TRUJILLO  JUZGADO DE FAMILIA    32   
1       CALLAO                CALLAO    CALLAO  JUZGADO DE FAMILIA    40   
2         LIMA  LURIGANCHO - CHOSICA      LIMA               OTROS    28   
3     AREQUIPA                 OCOÑA    CAMANA               OTROS    55   
4          ICA               PARCONA       ICA  JUZGADO DE FAMILIA    21   

     EST_CIVIL FEC_REGISTRO              MODALIDAD NIVEL_EDUCATIVO  \
0  CONVIVIENTE   2016-01-01    MALTRATO SIN LESION        SUPERIOR   
1   SOLTERO(A)   2016-01-02    MALTRATO SIN LESION      SECUNDARIA   
2  CONVIVIENTE   2016-01-03    MALTRATO SIN LESION      SECUNDARIA   
3   SOLTERO(A)   2016-01-03  VIOLENCIA PSICOLOGICA        PRIMARIA   
4   SOLTERO(A)   2016-01-04       VIOLENCIA FISICA      SECUNDARIA   

             OCUPACION SEXO SIT_PERSONA  
0        TECNICO DE EP    M  DENUNCIADO  
1          CESANTE PNP    M  DENUNCIAD

In [25]:
# Se llena el resto de los valores perdidos con la moda
for c in object_columns:
    mode = datalake[c].mode()[0]
    datalake[c] = datalake[c].fillna(mode)
missing_values()

Número de registros: 406431
Valores perdidos: 0


##### Separando fecha en año, mes, día y día de la semana

In [30]:
datalake['FEC_REGISTRO_ANIO'] = datalake['FEC_REGISTRO'].apply(lambda x: int(x[:4]))
datalake['FEC_REGISTRO_MES'] = datalake['FEC_REGISTRO'].apply(lambda x: int(x[5:7]))
datalake['FEC_REGISTRO_DIA'] = datalake['FEC_REGISTRO'].apply(lambda x: int(x[8:]))
datalake['FEC_REGISTRO_DIA_SEM'] = datalake['FEC_REGISTRO'].apply(lambda x: pd.to_datetime(x).weekday())

In [31]:
datalake.head()

,DPTO_CIA,DIST_CIA,PROV_CIA,DERIVADA_FISCALIA,EDAD,EST_CIVIL,FEC_REGISTRO,MODALIDAD,NIVEL_EDUCATIVO,OCUPACION,SEXO,SIT_PERSONA,FEC_REGISTRO_ANIO,FEC_REGISTRO_MES,FEC_REGISTRO_DIA,FEC_REGISTRO_DIA_SEM
0,LA LIBERTAD,HUANCHACO,TRUJILLO,JUZGADO DE FAMILIA,32,CONVIVIENTE,2016-01-01,MALTRATO SIN LESION,SUPERIOR,TECNICO DE EP,M,DENUNCIADO,2016,1,1,4
1,CALLAO,CALLAO,CALLAO,JUZGADO DE FAMILIA,40,SOLTERO(A),2016-01-02,MALTRATO SIN LESION,SECUNDARIA,CESANTE PNP,M,DENUNCIADO,2016,1,2,5
2,LIMA,LURIGANCHO - CHOSICA,LIMA,OTROS,28,CONVIVIENTE,2016-01-03,MALTRATO SIN LESION,SECUNDARIA,JIBILADO,F,DENUNCIADO,2016,1,3,6
3,AREQUIPA,OCOÑA,CAMANA,OTROS,55,SOLTERO(A),2016-01-03,VIOLENCIA PSICOLOGICA,PRIMARIA,OPERADOR DE MONTAJE,M,DENUNCIADO,2016,1,3,6
4,ICA,PARCONA,ICA,JUZGADO DE FAMILIA,21,SOLTERO(A),2016-01-04,VIOLENCIA FISICA,SECUNDARIA,BARMAT,F,DENUNCIADO,2016,1,4,0


#### Transformación de atributos categóricos a numéricos

In [34]:
datalake.drop(['FEC_REGISTRO'], axis=1, inplace=True)
print(datalake.info())
print(show_types())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406431 entries, 0 to 406430
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   DPTO_CIA              406431 non-null  object
 1   DIST_CIA              406431 non-null  object
 2   PROV_CIA              406431 non-null  object
 3   DERIVADA_FISCALIA     406431 non-null  object
 4   EDAD                  406431 non-null  int64 
 5   EST_CIVIL             406431 non-null  object
 6   MODALIDAD             406431 non-null  object
 7   NIVEL_EDUCATIVO       406431 non-null  object
 8   OCUPACION             406431 non-null  object
 9   SEXO                  406431 non-null  object
 10  SIT_PERSONA           406431 non-null  object
 11  FEC_REGISTRO_ANIO     406431 non-null  int64 
 12  FEC_REGISTRO_MES      406431 non-null  int64 
 13  FEC_REGISTRO_DIA      406431 non-null  int64 
 14  FEC_REGISTRO_DIA_SEM  406431 non-null  int64 
dtypes: int64(5), obje

In [37]:
def factorize(col):
    datalake[col] = pd.factorize(datalake[col])[0]
for col in object_columns:
    factorize(col)

In [40]:
show_types()

object
 []
int64
 ['DPTO_CIA', 'DIST_CIA', 'PROV_CIA', 'DERIVADA_FISCALIA', 'EDAD', 'EST_CIVIL', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'SEXO', 'SIT_PERSONA', 'FEC_REGISTRO_ANIO', 'FEC_REGISTRO_MES', 'FEC_REGISTRO_DIA', 'FEC_REGISTRO_DIA_SEM']


In [42]:
datalake.to_csv('./datalake/datalake_preprocessed.csv', index=False)